In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "pharmacy.pdf"
loader = PyPDFLoader(file_path)
data= loader.load()


c:\Users\kadii\miniconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(data)

94

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(data)

In [4]:
print(f"Number of documents after chunking: {len(docs)}")

Number of documents after chunking: 673


In [5]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [6]:
load_dotenv()

True

In [7]:
EMBED_MODEL = "models/gemini-embedding-001"

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


embeddings = GoogleGenerativeAIEmbeddings(model=EMBED_MODEL)
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [11]:
from langchain_community.vectorstores import FAISS


In [12]:
vectorstore = FAISS.from_documents(docs, embedding=embeddings)

In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [14]:
retrieved_docs = retriever.invoke("anti-tnf hangi hekim tarafından kullanılır?")

In [15]:
len(retrieved_docs)

5

In [29]:
print(retrieved_docs[4].page_content)

(1) Fistülize, şiddetli veya aktif luminal Crohn hastalığında;  en az bir anti TNF tedavisine rağmen hastalığı kontrol 
edilemeyen hastalarda, ilacın başlangıç tedavileri sonrasında idame dozunun sekiz haftada bir 300 mg olması koşuluyla,  ilacın 
kullanım dozu ve kullanım süresini belirten 4 ay süreli sağlık kurulu raporuna dayanılarak tedaviye başlanır. Tedavinin 4 üncü 
ayında tedaviye cevap alınamaması (hastalığa yanıt kriteri: Crohn Hastalık Aktivite İndeksi’ nde 70 puanlık düşüşün 
olmaması) durumunda tedavi kesilir. Crohn Hastalık Aktivite İndeksi’ nde 70 puan üzerinde düşüş olması halinde, ilacın dozu 
ve süresinin belirtildiği 6 ay süreli sağlık kurulu raporuna dayanılarak tedaviye devam edilebilir. Sağlık kurulu raporunda, 
gastroenteroloji, genel cerra hi uzman hekimlerinden en az biri yer almalıdır. Sağlık kurulu raporlarına dayanılarak, bu 
hekimler ile iç hastalıkları uzman hekimlerince reçete edilir ve sağlık kurum/kuruluşlarında klinik koşullarda uygulanır.


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

models = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-lite",  temperature=0.3, 
max_tokens=500)

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv


embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vectordb = Chroma.from_documents(docs, embeddings)

retriever = vectordb.as_retriever()


In [44]:
system_prompt = (
    "Sen eczacılık alanında uzman, SUT dokümanına göre cevap veren bir asistansın. "
    "Görevin, sadece sana verilen SUT PDF'inden elde edilen bilgilere dayanarak "
    "kullanıcının sorularını yanıtlamaktır. "
    "Aşağıdaki context parçalarını kullanarak sorulara cevap ver. "
    "Eğer cevap SUT dokümanında yoksa 'Bu konuda SUT dokümanında bilgi bulamıyorum.' de "
    "ve uydurma yapma. "
    "Cevapları Türkçe ver ve mümkün olduğunca kısa ve net yaz (en fazla 3-4 cümle)."
    "\n\n"
    "{context}"
)

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser



system_prompt = ("You are an assistant for question-answering tasks."
"Use the provided context to answer the question."
"If you don't know the answer, just say I don't know"
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.3,
    max_tokens=500,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

output_parser = StrOutputParser()


In [22]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

output_parser = StrOutputParser()

question_answering_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(), 
    }
    | prompt
    | llm
    | output_parser
)


In [23]:
question_answering_chain.invoke("Rapor kontrolü nasıl yapılır?"
)



'Rapor kontrolü, raporun kalitesini, doğruluğunu ve etkinliğini sağlamak için yapılan bir inceleme sürecidir. Bu süreç, raporun amacına ulaşıp ulaşmadığını, sunulan bilgilerin güvenilirliğini ve dilbilgisi ile imla hatalarını kontrol etmeyi kapsar.\n\nRapor kontrolünü yapmak için izlenebilecek adımlar şunlardır:\n\n1.  **Amacı Anlama:** Raporun neden yazıldığını ve kimin için hazırlandığını net bir şekilde anlamak. Raporun hedef kitlesi ve amacına uygun olup olmadığını değerlendirin.\n\n2.  **İçeriği Gözden Geçirme:**\n    *   **Doğruluk ve Güvenilirlik:** Sunulan tüm verilerin, istatistiklerin ve bilgilerin doğru ve güvenilir kaynaklardan alındığından emin olun. Gerekirse kaynakları teyit edin.\n    *   **Kapsamlılık:** Raporun, belirlenen amaca hizmet edecek kadar tüm ilgili bilgileri içerip içermediğini kontrol edin. Eksik veya gereksiz bilgi olup olmadığını değerlendirin.\n    *   **Mantıksal Akış ve Yapı:** Raporun bölümlerinin birbiriyle tutarlı ve mantıksal bir sırada ilerlediği

In [24]:
def ask(question: str):
    answer = question_answering_chain.invoke(question)
    print("🔹 Soru :", question)
    print("🔹 Cevap:", answer)
    return answer

# Test
ask("Anti-TNF hangi hekim tarafından kullanılır?")


🔹 Soru : Anti-TNF hangi hekim tarafından kullanılır?
🔹 Cevap: Anti-TNF tedavisini romatologlar, gastroenterologlar ve dermatologlar gibi ilgili uzmanlık alanlarındaki hekimler kullanır. Bu ilaçlar genellikle romatoid artrit, Crohn hastalığı, ülseratif kolit ve sedef hastalığı gibi inflamatuar hastalıkların tedavisinde kullanılır.


'Anti-TNF tedavisini romatologlar, gastroenterologlar ve dermatologlar gibi ilgili uzmanlık alanlarındaki hekimler kullanır. Bu ilaçlar genellikle romatoid artrit, Crohn hastalığı, ülseratif kolit ve sedef hastalığı gibi inflamatuar hastalıkların tedavisinde kullanılır.'